<a href="https://colab.research.google.com/github/RishabhGupta34/NLP_MODELS/blob/master/Bag_Of_Words_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"your_username","key":"your_key"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!kaggle competitions download -c word2vec-nlp-tutorial

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 85.2MB/s]
 62% 16.0M/26.0M [00:00<00:00, 69.7MB/s]
100% 26.0M/26.0M [00:00<00:00, 103MB/s] 
 40% 5.00M/12.6M [00:00<00:00, 29.5MB/s]
100% 12.6M/12.6M [00:00<00:00, 50.1MB/s]
 39% 5.00M/13.0M [00:00<00:00, 35.1MB/s]
100% 13.0M/13.0M [00:00<00:00, 63.4MB/s]


In [0]:
!ls {/content}/competitions/word2vec-nlp-tutorial

labeledTrainData.tsv.zip  testData.tsv.zip
sampleSubmission.csv	  unlabeledTrainData.tsv.zip


In [0]:
import os
import pickle
import zipfile
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import  RandomForestClassifier
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
zip_ref = zipfile.ZipFile('{/content}/competitions/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [0]:
zip_ref = zipfile.ZipFile('{/content}/competitions/word2vec-nlp-tutorial/testData.tsv.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
train_data=pd.read_csv('labeledTrainData.tsv',delimiter='\t',quoting=3)
test_data=pd.read_csv('testData.tsv',delimiter='\t',quoting=3)

In [0]:
def clean_text(data):
  cleaned_text=[]
  all_stop=stopwords.words('english')   #list of stopwords
  ss=SnowballStemmer("english")         #list of stemmer
  for i in range(data.shape[0]):
    soup=BeautifulSoup(data[i])            
    html_text=soup.get_text().lower()   #for removing html tags 
    slash=re.sub('[\\\\]',"",html_text) # for removing backward slash
    quotes_text=re.sub('[\"¨]'," <QUOT> ",slash) # for removing double quotes
    com=re.sub('[,]'," <COM> ",quotes_text) # for removing comma
    apos=re.sub("[']"," <APOS> ",com) # for removing apostrophe
    dott=re.sub('\.+'," <DOT> ",apos)  # for removing full stop
    exc=re.sub('!+'," <EXC> ",dott)  # for removing exclamation
    num=re.sub("\d+"," <NUM> ",exc)  # for removing number
    brac=re.sub('[()]+'," <BRAC> ",num)  # for removing brackets
    ques=re.sub("\?+"," <QUES> ",brac)  # for removing question mark
    split_text=ques.split()  # for splitting the text
    stop_text=[i for i in split_text if i not in all_stop]  # for removing stopwords
    stem_text=[ss.stem(i) for i in stop_text]  # for stemming each word
    final_text=[i for i in stem_text if len(i)>=2]  # for removing words with length 1
    cleaned_text.append(" ".join(final_text)) # converting the list of cleaned words into a single sentence
  return np.array(cleaned_text)

In [0]:
cleaned_train_reviews=clean_text(train_data.review)
cleaned_test_reviews=clean_text(test_data.review)

In [0]:
print(cleaned_train_reviews.shape,cleaned_test_reviews.shape)

(25000,) (25000,)


In [0]:
tf=TfidfVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words = None,ngram_range=(1,3),max_features =10000)

In [0]:
train=tf.fit_transform(cleaned_train_reviews)
test=tf.transform(cleaned_test_reviews)

In [0]:
train

<25000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3400361 stored elements in Compressed Sparse Row format>

In [0]:
test

<25000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3320965 stored elements in Compressed Sparse Row format>

In [0]:
len(tf.get_feature_names())

10000

In [0]:
rc=RandomForestClassifier(150)
rc.fit(train,train_data.sentiment)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
filename = 'drive/bag_of_words/finalized_model.sav'
pickle.dump(rc, open(filename, 'wb'))

In [0]:
rc = pickle.load(open(filename, 'rb'))

In [0]:
test_prediction=rc.predict(test)

In [0]:
output = pd.DataFrame( data={"id":test_data["id"], "sentiment":test_prediction} )

In [0]:
output.shape

(25000, 2)

In [0]:
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",0
3,"""7186_2""",1
4,"""12128_7""",1


In [0]:
output.to_csv( "drive/Bag_of_Words_model.csv", index=False, quoting=3 )